<a href="https://colab.research.google.com/github/AkiSugi/sample_AnimeGanV2/blob/main/AnimeGANv2_0518.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.はじめに  
AnimeGanの改良版であるAinmeGanv2をColaboratoryで動作させるために提供されているファイルを若干修正した  

スタイル生成のみ2021/5/19に動作確認を行った  

generator_Hayao_weight_lite は、Failed to find a checkpoint エラー  

GitHub AnimeGANv2  
https://github.com/TachibanaYoshino/AnimeGANv2  

AnimeGANv2 Projectページ  
https://tachibanayoshino.github.io/AnimeGANv2/  



 AnimeGANv2の改善の方向性としては、主に以下の4点である   
 1. 生成された画像に含まれる高周波のアーチファクトの問題を解決
 2. 学習が容易で、論文にある効果を直接実現できる

 3. ジェネレータネットワークのパラメータ数をさらに減らす。(generator size: 8.17 Mb), lite versionでは、ジェネレータモデルがより小さくなっている。

 4. BDムービーから取得した高品質の画像を学習に使用する。

# 1.事前準備  

In [ ]:
!git clone https://github.com/TachibanaYoshino/AnimeGANv2

スタイルを適用する画像ファイルを格納用 original フォルダを作成  

In [ ]:
%cd /content/AnimeGANv2
!mkdir original

# 2.初期化  
最初に実行するとき、あるいは、ランタイムを再起動した場合に、以下のCellから実行すること  

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

モジュールの読み込みと関数の定義  
 with tf.variable_scope("generator", reuse=tf.AUTO_REUSE): オリジナルでは、False  

---

 読み込み時に縦横32の倍数になるようリサイズを行う  
 256,256は最小サイズ  

---

 if_adjust_brightness がTrueの場合、生成された画像の明るさを元の画像を参考に調整を行う  
  OpenCVで輝度画像を作成し、平均値を求め、同じ値になるよう補正している  
  255のClip処理有  

In [ ]:
#%cd /content/AnimeGANv2

import os
import time
import numpy as np

from tools.utils import *
from glob import glob

from net import generator,generator_lite

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def stats_graph(graph):
    flops = tf.profiler.profile(graph, options=tf.profiler.ProfileOptionBuilder.float_operation())
    # params = tf.profiler.profile(graph, options=tf.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    print('FLOPs: {}'.format(flops.total_float_ops))

def test(checkpoint_dir, style_name, test_files, if_adjust_brightness, img_size=[256,256]):
    # tf.reset_default_graph()

    result_dir = 'results/'+style_name
    check_folder(result_dir)

    test_real = tf.placeholder(tf.float32, [1, None, None, 3], name='test')

    with tf.variable_scope("generator", reuse=tf.AUTO_REUSE):
        if 'lite' in checkpoint_dir:
            Anime_generated = generator_lite.G_net(test_real).fake
        else:
            Anime_generated = generator.G_net(test_real).fake

    saver = tf.train.Saver()

    gpu_options = tf.GPUOptions(allow_growth=True)
    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, gpu_options=gpu_options)) as sess:
        # tf.global_variables_initializer().run()
        # load model
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  # checkpoint file information
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)  # first line
            saver.restore(sess, os.path.join(checkpoint_dir, ckpt_name))
            print("\nSuccess to read {}\n".format(ckpt_name))
        else:
            print("\nFailed to find a checkpoint")
            return None

        # stats_graph(tf.get_default_graph())

        begin = time.time()
        for sample_file in test_files :
            print('Processing image: ' + sample_file)
            sample_image = np.asarray(load_test_data(sample_file, img_size))
            fname = os.path.basename(sample_file)
            save_path = os.path.join(result_dir,'{0}'.format('anime_'+style_name+'_'+fname))

            anime_img = sess.run(Anime_generated, feed_dict = {test_real : sample_image})
            anime = inverse_transform(anime_img.squeeze())

            # Trueを指定
            if if_adjust_brightness:
              org_img = read_img(sample_file)
              anime_adj = adjust_brightness_from_src_to_dst(anime, org_img)
              print("save to " + save_path)
              cv2.imwrite(save_path, cv2.cvtColor(anime_adj, cv2.COLOR_BGR2RGB))
              print(anime_adj.shape)
            else:
              print("save to " + save_path)
              cv2.imwrite(save_path, cv2.cvtColor(anime, cv2.COLOR_BGR2RGB))
              anime_img = anime

        end = time.time()
        print(f'\nTotal time: {end-begin} s')
        print(f'Average time : {(end-begin)/len(test_files)} s')

    return anime_img

# 3.Animeスタイルを適用した画像の生成  
originalフォルダーの画像は、一括でスタイル適用された画像が生成される    
画像を追加した場合、再度このcellから実行するだけでよい

スタイルを適用させる画像のサイズが大きいとGPUのリソースの枯渇でエラーとなる可能性がある  
2016x1512はOK 12秒    
4032x3024はエラー  

In [ ]:
%cd /content/AnimeGANv2

style = 'Hayao'
#style = 'Paprika'
#style = 'Shinkai'

ckpoint = 'checkpoint/generator_'+ style + '_weight'
#ckpoint = 'checkpoint/generator_Hayao_weight_lite'

org_files = glob('{}/*.*'.format('original'))
if len(org_files) > 0:
  st_img = test(ckpoint, style, org_files, True)
else:
  print("\nNo Image File")

# 4.ユーティリティ
/cotentの.jpgのファイルを縮小して、originalへ保存  



In [ ]:
import cv2
MAXSIZE=1280

org_files = glob('{}/*.jpg'.format('/content'))
print(org_files)

for file in org_files :
    img = cv2.imread(file)
    print(img.shape)
    h0 = img.shape[0]
    w0 = img.shape[1]

    if w0 > h0 :
        if w0 > MAXSIZE :
            w1 = MAXSIZE
            h1 = int(h0 * MAXSIZE / w0)
            img = cv2.resize(img, (w1, h1))
    else :
        if h1 > MAXSIZE :
            h1 = MAXSIZE
            w1 = int(w0 * MAXSIZE / h0)
            img = cv2.resize(img, (w1, h1))

    print (img.shape)
    fname = os.path.basename(file)
    save_path = os.path.join('/content/AnimeGANv2/original','{0}'.format('s_'+fname))
    print(save_path)
    cv2.imwrite(save_path, img)

styleデータを結果をすべてダウンロード  
複数のファイルのダウンロードの許可のダイアログに注意  

In [ ]:
from google.colab import files

res_path = '/content/AnimeGANv2/results/'+style
org_files = glob('{}/*.*'.format(res_path))
print(org_files)
for file in org_files :
    files.download(file)

In [ ]:
#!rm /content/AnimeGANv2/original/*.*

!rm /content/AnimeGANv2/results/Hayao/*.*